because avenue didn't support recursion or any of these fancypants things. 

sigh.

get off my lawn.

anyway. a little widget to test mapreduce with pooling (not threaded) for integration into a local simhash indexer (to test things without needing a database).

In [1]:
import sys
from multiprocessing import Pool
from uuid import uuid4  
import re
from collections import defaultdict

WORKERS = 5

def generate_uuid():
    return str(uuid4()).split('-')[0]

def _map(arr):
    # count by first char
    
    mapping = defaultdict(lambda: 0)
    for a in arr:
        mapping[a[0]] += 1
        
    return dict(mapping)

def _partition(items):
    # chunk into bits
    cut = int(round(len(items) / WORKERS + 0.5))
    i = 0
    while i < len(items):
        yield items[i:1+cut]
        i += cut

def _reduce(sub_mappings):
    r = defaultdict(lambda: 0)
    for sub in sub_mappings:
        for ch, count in sub.iteritems():
            r[ch] += count
        
    return sorted(r.items(), key=lambda t: (-t[1], t[0]))
  
def chunk(arr, r):
    for i in xrange(0, len(arr), r):
        yield arr[i:i+r]

In [4]:
big_list = [generate_uuid() for i in xrange(0, 1000)]

pool = Pool(processes=WORKERS,)

partitioned_list = list(_partition(big_list))

# get uuids starting with a digit
sub_maps = pool.map(_map, partitioned_list)

_reduce(sub_maps)
# # reduce to a histogram effectively
#counts = pool.map(_reduce, sub_maps.items())

#print counts

[('a', 18),
 ('7', 17),
 ('3', 16),
 ('c', 16),
 ('1', 15),
 ('9', 15),
 ('2', 14),
 ('d', 13),
 ('4', 12),
 ('0', 11),
 ('5', 11),
 ('8', 11),
 ('6', 10),
 ('e', 9),
 ('f', 8),
 ('b', 7)]